# CDC WONDER API
The US CDC makes a set of health-related databases available at https://wonder.cdc.gov, as the CDC Wide-ranging ONline Data for Epidemiologic Research or CDC WONDER.

Each database has an XML web service API endpoint, with some common elements described in the [documentation](https://wonder.cdc.gov/wonder/help/WONDER-API.html). But there are not data dictionaries available for the fields and codes of each database, nor a description of the XML format.

Below is a python example of calling the API from Python for cancer incidence data (database D144).  Thanks to [alipphardt's IPython notebook](https://github.com/alipphardt/cdc-wonder-api) showing details for D76 for inspiration and additional documentation.

Some precomputed data tables for the cancer incidence data are available at https://www.cdc.gov/cancer/uscs/dataviz/download_data.htm

In [ ]:
from lxml import etree
import pandas as pd
import requests

As described in the docs, the endpoint receives two POST parameters, `xmlrequest` and `accept_datause_restrictions`.
The XML request is a simple key-value pair collection:
```xml
<request-parameters>
    <parameter>
        <name>...</name>
        <value>...</value>
    </parameter>
    ...
</request-parameters>
```

In [ ]:
def make_request_xml(request_params):
    root = etree.Element('request-parameters')
    for name, value in request_params.items():
        param = etree.SubElement(root, 'parameter')
        etree.SubElement(param, 'name').text = name
        etree.SubElement(param, 'value').text = value
    return etree.ElementTree(root)

The keys, derived from looking at the WONDER web UI's API requests, are categorized by a few prefixes:
* `B_`: dimensions ("variables") to group by.
* `M_`: measures to show, disaggregated by each the grouping dimensions.
* `F_`/`I_`: simple and advanced form parameters.
* `O_`: other parameters.
* `V_`: filters for each dimension.
* unprefixed misc parameters

In [ ]:
req = make_request_xml({
    'B_1': 'D144.V10',
    'B_2': 'D144.V1',
    'B_3': 'D144.V2',
    'B_4': '*None*',  # 'D144.V4',
    'B_5': '*None*',  # 'D144.V6',
    'F_D144.V11': '*All*',
    'I_D144.V11': '*All* (The United States)',
    'M_1': 'D144.M1',
    'M_2': 'D144.M2',
    'M_40': 'D144.M40',
    'M_43': 'D144.M43',
    'O_PR': 'false',
    'O_V11_fmode': 'freg',
    'O_cancer': 'D144.V10',
    'O_javascript': 'on',
    'O_location': 'D144.V2',
    'O_precision': '3',
    'O_rate_per': '100000',
    'O_show_suppressed': 'true',
    'O_show_totals': 'true',
    'O_show_zeros': 'true',
    'O_stdpop': '201',
    'O_timeout': '900',
    'O_title': '',
    'V_D144.V10': '0',
    'V_D144.V11': '',
    'V_D144.V12': '*All*',
    'V_D144.V1': '*All*',
    'V_D144.V2': '*All*',
    'V_D144.V3': '*All*',
    'V_D144.V4': '*All*',
    'V_D144.V5': '*All*',
    'V_D144.V6': '*All*',
    'V_D144.V7': '*All*',
    'V_D144.V8': '*All*',
    'V_D144.V9': '*All*',
    'action-Send': 'Send',
    'dataset_code': 'D144',
    'dataset_id': 'D144',
    'dataset_label': 'United States and Puerto Rico Cancer Statistics, 1999-2015 Incidence',
    'dataset_vintage_latest': 'Cancer Incidence',
    'finder-stage-D144.V11': 'codeset',
    'saved_id': '',
    'stage': 'request',
    'accept_datause_restrictions': 'true',
})

In [ ]:
endpoint = 'https://wonder.cdc.gov/controller/datarequest/D144'
r = requests.post(endpoint, data={
    'request_xml': etree.tostring(req),
    'accept_datause_restrictions': 'true'
})

In [ ]:
r.raise_for_status()

The response is XML.

In [ ]:
resp = etree.ElementTree(etree.fromstring(r.text))

In [ ]:
with open('D144.xml', 'wb') as f:
    f.write(etree.tostring(resp, pretty_print=True))

The response includes a dataset section which includes, among other things, the codes for the requested measures:

In [ ]:
measures = {}
for measure in resp.findall('/dataset/measure'):
    measures[measure.attrib['code']] = measure.attrib['label']
measures

It also includes the codes and codelists for the dimensions (called "variables").  These also include grouping metadata, in this example Cancers (site), (geographic) Location, and Year and Demographics. In addition, dimensions which can provide values for `O_` fields indicate which fields (controls) they apply to.

In [ ]:
from collections import defaultdict
controls = defaultdict(list)
groups = {}
dimensions = {}
for variable in resp.findall('/dataset/variable'):
    if 'groupby-heading' in variable.attrib:
        heading = variable.attrib['groupby-heading']
        if heading != 'endgroup':
            groups[heading] = set()
        else:
            heading = None
    else:
        values = {}
        for value in variable.findall('./value'):
            values[value.attrib['code']] = value.attrib['label']
        groups[heading].add(variable.attrib['code'])
        dimensions[variable.attrib['code']] = {
            'name': variable.attrib['label'],
            'codelist': values
        }
        for control in variable.findall('other-param-control'):
            controls[control.attrib['code']].append(control.attrib['value'])
groups, controls, dimensions

The dataset section includes some descriptive metadata, such as citation, description, related URLs, and caveats for using the data.

In [ ]:
citation = resp.find('/dataset/documentation/citation').text
citation_url = resp.find('/dataset/documentation/citationURL').text
desc = resp.find('/dataset/documentation/about').text
data_url = resp.find('/dataset/documentation/dataurl').text
caveats={}
for caveat in resp.findall('/dataset/documentation/caveat'):
    caveats[caveat.attrib['code']] = caveat.find('text').text

The response data is a tabular structure, so we need to record the column identifiers in order.

In [ ]:
slice_variables = []
slice_measures = []
for variable in resp.findall('/response/request/byvariables/variable'):
    slice_variables.append(variable.attrib['code'])
for measure in resp.findall('/response/request/measure-selections/measure'):
    slice_measures.append(measure.attrib['code'])

columns_byname = {dimensions[dimension]['name']: dimension for dimension in slice_variables}
columns_byname.update({measures[measure]: measure for measure in slice_measures})
columns = list(columns_byname.keys())

The table consists of `r` rows containing `c` cells, which may contain `l` (labels? additional data).  
Numerical values are stored in the cell's `v` attribute, and textual values in its `l` label attribute.  
Categorical data may include code `cd`, and (unknown) `cf` attributes.

Per the caveats, the data may include `Missing` or `Suppressed` values, both of which I treat as null/NA below.

In [ ]:
data_table = []
for row in resp.findall('/response/data-table/r'):
    row_vals = []
    for cell in row.findall('c'):
        val = cell.attrib.get('l')
        if val is None:
            val = cell.attrib['v'].replace(',', '')
            if val == 'Missing' or val == 'Suppressed':
                val = None
            else:
                val = float(val)
        row_vals.append(val)
        # Check for row span
        span = cell.attrib.get('r')
        if span is not None:
            row_vals.append([] * (int(span)-1))
        # Check for combine labels
        l = cell.find('l')
        if l is not None:
            row_vals.append(float(l.attrib.get('v')))
    # The `l` child is missing on some rows; pad under-full rows with None.
    while len(row_vals) < len(columns):
        row_vals.append(None)
    if len(row_vals) > len(columns):
        raise RuntimeError("Too many columns")
    data_table.append(row_vals)
slice_df = pd.DataFrame(data=data_table,
             columns=columns)
slice_df.to_csv('slice.csv')

In [ ]:
slice_df

For fun, let's transform the dataset to [DSPL 2](https://google.github.io/dspl/dspl2-spec.html). This isn't useful for much yet, but you can try out the tools linked to at https://google.github.io/dspl .

In [ ]:
label_to_code = {}
for dim in slice_variables:
    label_to_code[dim] = {}
    for code, label in dimensions[dim]['codelist'].items():
        label_to_code[dim][label] = code

In [ ]:
from pathlib import Path
import csv
import json

try:
    Path('dataset').mkdir()
except:
    pass

dspl2 = {
    '@type': 'StatisticalDataset',
    'name': citation,
    'url': citation_url,
    'description': desc,
    'license': 'https://wonder.cdc.gov/datause.html',
    'distribution': {
        '@type': 'DataDownload',
        'url': data_url,
    },
    'dimension': [],
    'measure': [],
    'slice': [],
}
for code, label in measures.items():
    dspl2['measure'].append({
        '@type': 'StatisticalMeasure',
        '@id': f'#{code}',
        'name': label,
    })
for code, dimension in dimensions.items():
    dspl2['dimension'].append({
        '@type': 'CategoricalDimension',
        '@id': f'#{code}',
        'name': dimension['name'],
        'codeList': f'{code}.csv'
    })
    df = pd.DataFrame(list(dimension['codelist'].items()), columns=['codeValue', 'name'])
    df.to_csv(f'dataset/{code}.csv', index=False)
dspl2['slice'].append({
    '@type': 'DataSlice',
    'dimension': [f'#{dimension}' for dimension in slice_variables],
    'measure': [f'#{code}' for code in slice_measures],
    'data': 'slice.csv'
})
with open('dataset/slice.csv', 'w') as f:
    out = csv.writer(f)
    out.writerow([columns_byname[col] for col in slice_df.columns])
    for row in slice_df.itertuples(index=False):
        coded_row = []
        for dim,label in zip(slice_variables,row):
            coded_row.append(label_to_code[dim][label])
        coded_row.extend(row[len(slice_variables):])
        out.writerow(coded_row)
with open('dataset/cancer.json', 'w') as f:
    json.dump(dspl2, f)

Now let's do some simple visualization. Here, a small multiples chart showing the overall cancer rate by state, for each year from 2000 to 2015.

In [ ]:
import geopandas as gpd

In [ ]:
us = gpd.read_file('https://eric.clst.org/assets/wiki/uploads/Stuff/gz_2010_us_040_00_5m.json')
us = us.set_index('NAME')

In [ ]:
total = slice_df.groupby(['States', 'Year'])['Count','Population'].sum()
total['Rate'] = total['Count']/total['Population'] * 10000
total = total.reset_index().set_index('States')
total.index.name = 'NAME'

In [ ]:
mappable = us.join(total)

In [ ]:
import matplotlib.pyplot as plt
fig, axes = plt.subplots(5,3, figsize=(25,25))
for ax, year in zip(axes.ravel(), range(2000, 2016)):
    mappable[mappable['Year'] == str(year)].drop(['Alaska', 'Hawaii']).plot(
        column='Rate', cmap='Blues',
        vmin=mappable['Rate'].min(),
        vmax=mappable['Rate'].max(),
        ax=ax,
        legend=False)
    ax.set_title(f"{year}")
    ax.set_xticks([])
    ax.set_yticks([])
print("Cancer incidence rate by state")